In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
tf.keras.backend.clear_session()

In [6]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [7]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [8]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2008,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,0.808544,0.686768,0.746558,5.483335e+08,2.546644e+09,...,30.360011,952.583468,1388.122906,23.505311,16.064850,66.686091,3.455270,0.749040,8.011861,814.771000
1,2009,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,0.809433,0.689912,0.747506,5.302764e+08,2.679945e+09,...,31.066389,952.388003,1396.303897,23.414338,15.953541,66.658540,3.422201,0.748012,8.040524,693.327000
2,2010,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,0.809776,0.693020,0.748442,5.046425e+08,2.891930e+09,...,32.508565,952.362034,1428.326267,23.460429,15.531732,65.002039,3.316282,0.747440,8.003862,813.149000
3,2011,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,0.809583,0.696082,0.749364,4.854622e+08,3.091988e+09,...,34.904902,952.558014,1433.680808,23.635226,15.687149,64.896779,3.223002,0.747314,7.993144,923.248000
4,2012,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,0.808780,0.698913,0.750034,4.867661e+08,3.189506e+09,...,34.979340,952.759391,1421.572593,23.663605,15.955379,65.719662,3.178068,0.747266,6.360445,1008.624000
5,2013,5.452953e+07,2.278494e+06,16.940587,4.240902e+07,0.807595,0.702405,0.751453,5.225848e+08,3.093872e+09,...,33.811640,952.954665,1405.057523,23.757867,16.260264,66.470023,3.125032,0.748788,6.725372,1028.608000
6,2014,5.543864e+07,2.276794e+06,16.985287,4.261295e+07,0.805894,0.704772,0.752158,5.676413e+08,2.779340e+09,...,33.031265,953.098166,1382.443280,23.914606,16.608338,67.060540,3.088252,0.749938,7.077333,995.222143
7,2015,5.607449e+07,2.266815e+06,16.994005,4.270203e+07,0.803719,0.707587,0.752754,5.179392e+08,2.561731e+09,...,31.224126,953.111147,1377.219450,23.862880,16.783027,67.636795,3.072316,0.750599,7.357560,951.041314
8,2016,5.656732e+07,2.251674e+06,17.013733,4.279700e+07,0.809908,0.717698,0.756127,4.524144e+08,2.516859e+09,...,29.308459,952.984733,1353.708395,23.814967,16.659956,67.363296,3.049349,0.744326,7.332914,904.537543
9,2017,5.674497e+07,2.236955e+06,17.015525,4.281848e+07,0.806990,0.719489,0.755957,4.213689e+08,2.497710e+09,...,27.027249,953.036051,1260.888062,23.920271,16.556341,66.782526,2.972744,0.744629,7.012814,870.341000


In [9]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
0,4.895245e+07,2.250403e+06,16.533879,4.072450e+07,0.808544,0.686768,0.746558,5.483335e+08,2.546644e+09,1.874883e+10,...,869.834453,30.360011,952.583468,1388.122906,23.505311,16.064850,66.686091,3.455270,0.749040,8.011861
1,4.943099e+07,2.269266e+06,16.565699,4.085410e+07,0.809433,0.689912,0.747506,5.302764e+08,2.679945e+09,2.055015e+10,...,949.083048,31.066389,952.388003,1396.303897,23.414338,15.953541,66.658540,3.422201,0.748012,8.040524
2,5.089988e+07,2.290154e+06,16.701299,4.138190e+07,0.809776,0.693020,0.748442,5.046425e+08,2.891930e+09,2.342247e+10,...,1038.662223,32.508565,952.362034,1428.326267,23.460429,15.531732,65.002039,3.316282,0.747440,8.003862
3,5.224283e+07,2.294876e+06,16.813003,4.180160e+07,0.809583,0.696082,0.749364,4.854622e+08,3.091988e+09,2.604265e+10,...,1134.526356,34.904902,952.558014,1433.680808,23.635226,15.687149,64.896779,3.223002,0.747314,7.993144
4,5.345401e+07,2.288752e+06,16.887782,4.213702e+07,0.808780,0.698913,0.750034,4.867661e+08,3.189506e+09,2.708753e+10,...,1291.161206,34.979340,952.759391,1421.572593,23.663605,15.955379,65.719662,3.178068,0.747266,6.360445
5,5.452953e+07,2.278494e+06,16.940587,4.240902e+07,0.807595,0.702405,0.751453,5.225848e+08,3.093872e+09,2.523394e+10,...,1371.926397,33.811640,952.954665,1405.057523,23.757867,16.260264,66.470023,3.125032,0.748788,6.725372
6,5.543864e+07,2.276794e+06,16.985287,4.261295e+07,0.805894,0.704772,0.752158,5.676413e+08,2.779340e+09,1.954361e+10,...,1268.814714,33.031265,953.098166,1382.443280,23.914606,16.608338,67.060540,3.088252,0.749938,7.077333
7,5.607449e+07,2.266815e+06,16.994005,4.270203e+07,0.803719,0.707587,0.752754,5.179392e+08,2.561731e+09,1.327596e+10,...,1123.345627,31.224126,953.111147,1377.219450,23.862880,16.783027,67.636795,3.072316,0.750599,7.357560
8,5.656732e+07,2.251674e+06,17.013733,4.279700e+07,0.809908,0.717698,0.756127,4.524144e+08,2.516859e+09,1.011370e+10,...,1075.219903,29.308459,952.984733,1353.708395,23.814967,16.659956,67.363296,3.049349,0.744326,7.332914
9,5.674497e+07,2.236955e+06,17.015525,4.281848e+07,0.806990,0.719489,0.755957,4.213689e+08,2.497710e+09,8.456309e+09,...,1111.839984,27.027249,953.036051,1260.888062,23.920271,16.556341,66.782526,2.972744,0.744629,7.012814


In [10]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      693.327000
1      813.149000
2      923.248000
3     1008.624000
4     1028.608000
5      995.222143
6      951.041314
7      904.537543
8      870.341000
9      869.680000
10     902.144000
11    1038.045000
12    1092.056000
13    1152.060000
14            NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [11]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
0,-1.796747,-0.650239,-1.985110,-1.919451,0.518131,-1.517946,-1.602263,1.204752,-0.990234,0.166091,...,-1.925963,-0.338768,-0.908635,0.325154,-1.091330,-0.334797,0.493297,1.511319,0.860252,1.264582
1,-1.624414,0.229010,-1.795763,-1.743254,0.799491,-1.254064,-1.303980,0.820657,-0.546688,0.428911,...,-1.344018,-0.064876,-1.590134,0.466286,-1.454581,-0.619536,0.469052,1.363350,0.357086,1.315878
2,-1.095430,1.202637,-0.988869,-1.025725,0.908067,-0.993168,-1.009842,0.275392,0.158672,0.847995,...,-0.686213,0.494314,-1.680674,1.018708,-1.270544,-1.698561,-0.988622,0.889403,0.077345,1.250267
3,-0.611803,1.422742,-0.324167,-0.455159,0.846962,-0.736133,-0.719926,-0.132596,0.824346,1.230291,...,0.017744,1.423471,-0.997382,1.111080,-0.572583,-1.300989,-1.081249,0.472017,0.015931,1.231086
4,-0.175626,1.137319,0.120809,0.000837,0.592735,-0.498516,-0.509304,-0.104860,1.148829,1.382743,...,1.167958,1.452334,-0.295274,0.902200,-0.459269,-0.614833,-0.357135,0.270956,-0.007606,-1.690786
5,0.211695,0.659137,0.435031,0.370620,0.217752,-0.205323,-0.062964,0.657046,0.830615,1.112297,...,1.761039,0.999569,0.385559,0.617296,-0.082883,0.165091,0.303163,0.033641,0.737208,-1.037715
6,0.539088,0.579898,0.701020,0.647859,-0.320327,-0.006617,0.158537,1.615453,-0.215960,0.282052,...,1.003861,0.696986,0.885881,0.227174,0.542969,1.055494,0.822801,-0.130934,1.299438,-0.407846
7,0.768073,0.114776,0.752894,0.768952,-1.008723,0.229633,0.346091,0.558230,-0.940033,-0.632425,...,-0.064359,-0.003715,0.931139,0.137057,0.336428,1.502365,1.329890,-0.202240,1.623093,0.093648
8,0.945552,-0.590971,0.870290,0.898070,0.949764,1.078400,1.406722,-0.835563,-1.089341,-1.093812,...,-0.417760,-0.746497,0.490392,-0.268535,0.145116,1.187538,1.089218,-0.305007,-1.446069,0.049540
9,1.009527,-1.277063,0.880952,0.927264,0.026583,1.228721,1.353185,-1.495939,-1.153056,-1.335633,...,-0.148848,-1.631015,0.669314,-1.869792,0.565589,0.922481,0.578156,-0.647783,-1.298100,-0.523309


In [12]:
reshaped_train_input = dfToInputRNN(train_input)

In [13]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.8020364 , -1.2497368 ,  0.47606804,  0.62243838,
         -1.47954097,  1.23556063,  0.78546116, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.46480286, -0

In [14]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      693.327000
1      813.149000
2      923.248000
3     1008.624000
4     1028.608000
5      995.222143
6      951.041314
7      904.537543
8      870.341000
9      869.680000
10     902.144000
11    1038.045000
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [15]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1038.045
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [16]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
12,0.585267,-1.32885,0.291975,0.434228,-2.576211,1.160996,0.103862,1.108405,2.635823,-0.189959,...,1.207226,-0.26749,0.720445,0.131831,1.921027,-0.898095,-2.178339,-2.706799,-1.357455,-1.015947


In [17]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.58526692, -1.3288499 ,  0.29197484,  0.43422826,
         -2.57621081,  1.16099557,  0.10386236,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          1.20722572, -0

In [18]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1092.056
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [19]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [21]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[556835512, 2624484675, 1986301789, 3049008833, 4148706308, 1388735019, 1319923192, 1584072089, 2700281109, 2070722511, 2779910005, 2717929719, 1997179547, 3417865718, 258360680, 3880235901, 1378143325, 1103290572, 693239294, 3790663641, 624744034, 3476864432, 4086821416, 727031780, 1791214420, 2807399356, 997625908, 3641002499, 3776668606, 3366429230, 2260423269, 2034594362, 2692650801, 2034015566, 784188101, 3503789285, 1291356086, 19527333, 1527029941, 3302796863, 2230830405, 2595904246, 28218373, 125654577, 1900568794, 1277508370, 2836451015, 344532573, 309087618, 1369808669, 4190549739, 2046299044, 1262453377, 3749886967, 2683983628, 3505147864, 294743446, 1131355483, 2228901057, 765944622, 1781805924, 3747132036, 4140263336, 1883483627, 390945769, 3597099418, 4205438047, 4142723641, 3070146474, 3047883012, 1650363977, 2125156144, 1697797516, 1441658627, 1216027926, 727070184, 3968446708, 171892924, 2213812395, 2812697415, 1353474364, 2197850488, 436875837, 4046686100, 3504121074,

2023-09-08 18:11:26.434853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 18:11:26.470455: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 18:11:26.470636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 220.5147705078125
winner_seed: 556835512


Step: 1 ___________________________________________
loss: 169.95098876953125
winner_seed: 2624484675


Step: 2 ___________________________________________
loss: 182.7249755859375


Step: 3 ___________________________________________
loss: 154.773193359375
winner_seed: 3049008833


Step: 4 ___________________________________________
loss: 202.86045837402344


Step: 5 ___________________________________________
loss: 164.1757049560547


Step: 6 ___________________________________________
loss: 195.49609375


Step: 7 ___________________________________________
loss: 183.8276824951172


Step: 8 ___________________________________________
loss: 204.3270721435547


Step: 9 ___________________________________________
loss: 137.44862365722656
winner_seed: 2070722511


Step: 10 ___________________________________________
loss: 154.3365020751953


Step: 11 ___________________________________________
loss: 101.59464263916016
winner_seed: 2717929719

In [22]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 848995.0000
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 829719.6875
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 763671.9375
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 281581.2188
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 42935.7383
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 26752.9473
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 17632.7266
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 12721.5498
Epoch 9/10000
1/1 [==============================] - 0s 3ms/step - loss: 11676.3271
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 7649.9966
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 12381.4297
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss

1/1 [==============================] - 0s 4ms/step - loss: 2120.3801
Epoch 99/10000
1/1 [==============================] - 0s 3ms/step - loss: 2788.1804
Epoch 100/10000
1/1 [==============================] - 0s 3ms/step - loss: 2246.0530
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 1521.1548
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 1707.5894
Epoch 103/10000
1/1 [==============================] - 0s 3ms/step - loss: 1366.2100
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 1118.1819
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 812.4798
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 1901.3605
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 1347.5717
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 1661.9430
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 1391.618

1/1 [==============================] - 0s 4ms/step - loss: 1440.9664
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 1249.3344
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 1791.8864
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 1848.1295
Epoch 198/10000
1/1 [==============================] - 0s 3ms/step - loss: 1936.2069
Epoch 199/10000
1/1 [==============================] - 0s 3ms/step - loss: 616.5034
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 2292.4331
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 1925.4196
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 869.1878
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 1601.8324
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 1200.3324
Epoch 205/10000
1/1 [==============================] - 0s 3ms/step - loss: 1296.892

1/1 [==============================] - 0s 4ms/step - loss: 2321.5005
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 737.6777
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 1435.6299
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 546.0037
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 718.2734
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 761.1022
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 1119.7264
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 1275.6367
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 1432.0399
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 550.1219
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 694.4753
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 815.1625
Epo

Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 551.5176
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 969.9936
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 572.6147
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 1261.8126
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 1374.6088
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 832.7610
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 1817.3177
Epoch 394/10000
1/1 [==============================] - 0s 5ms/step - loss: 1264.4471
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 1939.5615
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 1231.4741
Epoch 397/10000
1/1 [==============================] - 0s 5ms/step - loss: 1914.1959
Epoch 398/10000
1/1 [==============================] - 0s 4ms/step - 

...

1/1 [==============================] - 0s 4ms/step - loss: 1634.8527
Epoch 2113/10000
1/1 [==============================] - 0s 4ms/step - loss: 646.9729
Epoch 2114/10000
1/1 [==============================] - 0s 3ms/step - loss: 1881.7345
Epoch 2115/10000
1/1 [==============================] - 0s 3ms/step - loss: 1964.3461
Epoch 2116/10000
1/1 [==============================] - 0s 3ms/step - loss: 1014.7346
Epoch 2117/10000
1/1 [==============================] - 0s 3ms/step - loss: 753.5439
Epoch 2118/10000
1/1 [==============================] - 0s 3ms/step - loss: 817.8624
Epoch 2119/10000
1/1 [==============================] - 0s 3ms/step - loss: 709.7889
Epoch 2120/10000
1/1 [==============================] - 0s 3ms/step - loss: 2445.6416
Epoch 2121/10000
1/1 [==============================] - 0s 4ms/step - loss: 914.0628
Epoch 2122/10000
1/1 [==============================] - 0s 3ms/step - loss: 616.4418
Epoch 2123/10000
1/1 [==============================] - 0s 3ms/step - loss: 1

1/1 [==============================] - 0s 3ms/step - loss: 600.3000
Epoch 2208/10000
1/1 [==============================] - 0s 3ms/step - loss: 1109.8138
Epoch 2209/10000
1/1 [==============================] - 0s 4ms/step - loss: 986.0052
Epoch 2210/10000
1/1 [==============================] - 0s 4ms/step - loss: 442.5094
Epoch 2211/10000
1/1 [==============================] - 0s 3ms/step - loss: 1325.7914
Epoch 2212/10000
1/1 [==============================] - 0s 4ms/step - loss: 400.5350
Epoch 2213/10000
1/1 [==============================] - 0s 3ms/step - loss: 396.8427
Epoch 2214/10000
1/1 [==============================] - 0s 3ms/step - loss: 2510.7305
Epoch 2215/10000
1/1 [==============================] - 0s 4ms/step - loss: 1009.2806
Epoch 2216/10000
1/1 [==============================] - 0s 3ms/step - loss: 529.5383
Epoch 2217/10000
1/1 [==============================] - 0s 4ms/step - loss: 935.6788
Epoch 2218/10000
1/1 [==============================] - 0s 4ms/step - loss: 76

1/1 [==============================] - 0s 3ms/step - loss: 1015.3776
Epoch 2304/10000
1/1 [==============================] - 0s 4ms/step - loss: 568.7823
Epoch 2305/10000
1/1 [==============================] - 0s 3ms/step - loss: 1067.2919
Epoch 2306/10000
1/1 [==============================] - 0s 4ms/step - loss: 1110.0111
Epoch 2307/10000
1/1 [==============================] - 0s 4ms/step - loss: 1495.7911
Epoch 2308/10000
1/1 [==============================] - 0s 4ms/step - loss: 1047.5260
Epoch 2309/10000
1/1 [==============================] - 0s 4ms/step - loss: 1213.8077
Epoch 2310/10000
1/1 [==============================] - 0s 4ms/step - loss: 1313.5197
Epoch 2311/10000
1/1 [==============================] - 0s 4ms/step - loss: 1114.4987
Epoch 2312/10000
1/1 [==============================] - 0s 4ms/step - loss: 938.0468
Epoch 2313/10000
1/1 [==============================] - 0s 4ms/step - loss: 1140.9425
Epoch 2314/10000
1/1 [==============================] - 0s 3ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 704.1645
Epoch 2399/10000
1/1 [==============================] - 0s 4ms/step - loss: 736.7297
Epoch 2400/10000
1/1 [==============================] - 0s 4ms/step - loss: 909.6250
Epoch 2401/10000
1/1 [==============================] - 0s 4ms/step - loss: 931.6846
Epoch 2402/10000
1/1 [==============================] - 0s 4ms/step - loss: 1118.9740
Epoch 2403/10000
1/1 [==============================] - 0s 4ms/step - loss: 961.1011
Epoch 2404/10000
1/1 [==============================] - 0s 4ms/step - loss: 1030.1642
Epoch 2405/10000
1/1 [==============================] - 0s 4ms/step - loss: 715.2269
Epoch 2406/10000
1/1 [==============================] - 0s 4ms/step - loss: 1520.3027
Epoch 2407/10000
1/1 [==============================] - 0s 4ms/step - loss: 1260.8855
Epoch 2408/10000
1/1 [==============================] - 0s 4ms/step - loss: 1014.5798
Epoch 2409/10000
1/1 [==============================] - 0s 4ms/step - loss: 7

Epoch 2494/10000
1/1 [==============================] - 0s 4ms/step - loss: 650.8412
Epoch 2495/10000
1/1 [==============================] - 0s 4ms/step - loss: 2134.9167
Epoch 2496/10000
1/1 [==============================] - 0s 4ms/step - loss: 1352.6022
Epoch 2497/10000
1/1 [==============================] - 0s 4ms/step - loss: 1945.5900
Epoch 2498/10000
1/1 [==============================] - 0s 5ms/step - loss: 398.1948
Epoch 2499/10000
1/1 [==============================] - 0s 4ms/step - loss: 1167.2894
Epoch 2500/10000
1/1 [==============================] - 0s 4ms/step - loss: 820.5259
Epoch 2501/10000
1/1 [==============================] - 0s 4ms/step - loss: 910.4956
Epoch 2502/10000
1/1 [==============================] - 0s 4ms/step - loss: 1342.4354
Epoch 2503/10000
1/1 [==============================] - 0s 4ms/step - loss: 870.2021
Epoch 2504/10000
1/1 [==============================] - 0s 4ms/step - loss: 969.2607
Epoch 2505/10000
1/1 [==============================] - 0s 4

In [23]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 595ms/step
[1092.056](test_target) - [1039.9359](prediction) = 52.12008691406254


In [24]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.04772656980416987

In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
0,-0.975936,-1.203430,-0.914858,-0.922677,-1.361786,-1.227061,-1.227592,1.148205,-1.122043,-1.121418,...,-1.199016,-1.064331,1.406054,-0.929666,1.219289,0.935499,0.724528,0.967433,1.323407,-0.437633
1,-0.398407,-0.041572,-0.476529,-0.466832,0.350506,0.004646,0.005713,0.140893,-0.184474,-0.185492,...,-0.049930,-0.274308,-0.571654,-0.458093,-1.230128,0.450745,0.689541,0.409623,-0.229909,1.383444
2,1.374343,1.245001,1.391386,1.389509,1.011279,1.222415,1.221878,-1.289098,1.306518,1.306910,...,1.248946,1.338639,-0.834400,1.387758,0.010839,-1.386244,-1.414069,-1.377056,-1.093497,-0.945811


train_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
0,693.327
1,813.149
2,923.248


test_input:


...

,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso do Sul - IDH Longevidade,Mato Grosso do Sul - IDH Educacao,Mato Grosso do Sul - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - Desemprego
12,0.585267,-1.32885,0.291975,0.434228,-2.576211,1.160996,0.103862,1.108405,2.635823,-0.189959,...,1.207226,-0.26749,0.720445,0.131831,1.921027,-0.898095,-2.178339,-2.706799,-1.357455,-1.015947


test_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
12,1092.056


1/1 [==============================] - 1s 585ms/step
Error: 52.12008691406254




[916.1171875,
 992.2123413085938,
 429.8346252441406,
 989.5255126953125,
 951.9736328125,
 915.9549560546875,
 872.8728637695312,
 875.7666625976562,
 887.1614379882812,
 1039.9359130859375]

In [27]:
display(targets)
display(predictions)

[1008.624,
 1028.608,
 995.2221428571428,
 951.0413142857142,
 904.5375428571429,
 870.341,
 869.68,
 902.144,
 1038.045,
 1092.056]

[916.1171875,
 992.2123413085938,
 429.8346252441406,
 989.5255126953125,
 951.9736328125,
 915.9549560546875,
 872.8728637695312,
 875.7666625976562,
 887.1614379882812,
 1039.9359130859375]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

105.83980833217079

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.10956162778416152